### Load Data

In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

import torchvision
from torchvision import transforms
from torchvision import models
from torchvision.datasets import CIFAR10


# download training data
train_data = CIFAR10(root="./train/",
                     train=True,
                     download=True,
                     transform=None)

print(train_data)


### Accessing the Data

`How do you access the label?` <br>
<p> The label can be accessed with by getting the value of the [1] index of a train_data[i] tuple </p>

In [ ]:
label = train_data[16][1]
data_class = train_data.classes[train_data[16][1]]
print(f'Data Label: {label}')                       # print the label of the tuple
print(f'Class: {data_class}')    # print the class of the image

`What method is called when you index into a Dataset?`

The Dataset function __getitem__(self, index: int) is called. It takes a self-reference and integer as arguments to return a tuple containing the image and target class.

`Is CIFAR10 a class that is derived from the Dataset class?`

Yes, CIFAR10 is a subclass of Dataset that inherits the __getitem__ and __len__ methods from Dataset.

`Inheritance Tree`
<p> do this</p>

### Data Transforms

Pre-process the data before running through a neural net

In [ ]:
# TRAINING DATA

# taking mean and std values from the book
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                        std=(0.2023, 0.1994, 0.201)),
])

train_data_xform = CIFAR10(root="./train/",
                     train=True,
                     transform=train_transform)

data, label = train_data[16]
print("Training data without transforms: ")
print(data)
print(label)

data, label = train_data_xform[16]
print("Training data with transforms: ")
print(data)
print(label)



# TESTING DATA
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        (0.4914, 0.4822, 0.4465),
        (0.2023, 0.1994, 0.2010))
    ])

test_data = CIFAR10(root="./test/",
                     train=False,
                     transform=test_transform,
                     download=True)





`When you instantiate train_data the second time, with the transform, try without download=True. Look at the API. What does it say?`
<p>If download is set to 'True', the data is downloaded to the root directory, otherwise it will verify the data has been downloaded. If the 'download' option is not included and the Dataset is not found, it will cause a runtime error.</p>

`What is the difference between training and testing transforms? Training is supposed to ”see” more data variability and that is why we provide augmentations of the original data through transforms. Why do you think the test dataset has a different transform?`

The training transform has augmentations such as horizontal flips and random crops whereas the testing data is unaltered aside from normalization. Keeping the two sets seperated like this helps simulate real-world variabiltiy and makes the make generalizations about unseen data. Augmenting the testing data may lead to overly-optimistic results.

`Please do` <br>
`data, label = train_data[index] in both cases (with and without transforms).`<br>
`Why is your result different when you apply transforms?`

Before transforms are applied, the data is raw. When accessing train_data[index], metadata is returned such as the color mode, size, and address in memory. When the data is processed with a transform, it's changed into a tensor so that it is readable by a machine learning model.

### Data Batching

In [ ]:
# Dataloader does all the work of shuffling data between batches and training cycles (epochs)

train_loader = torch.utils.data.DataLoader(
    train_data_xform,
    batch_size = 16,
    shuffle = True)

# create data batches
data_batch, labels_batch = next(iter(train_loader))
print(data_batch.size())
# out: torch.Size([16, 3, 32, 32])

print(labels_batch.size())
test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size = 16,
    shuffle = False) # set shuffle to false for the testing data for repeatable results


### Model Design

In [ ]:
vgg16 = models.vgg16(pretrained=False)

# Print layers of the neural network
print("Neural Network Layers: ")
print(vgg16)

In [ ]:
# replace the Linear transformation layer with a new definition
vgg16.classifier[-1] = nn.Linear(4096, 10)

# not enough memory on laptop GPU
device = "cpu"

model = vgg16.to(device=device)

### Model Training

In [ ]:


print("Setting criterion...")
criterion = nn.CrossEntropyLoss()
print("Implementing stochastic gradient descent...")
optimizer = optim.SGD(model.parameters(),
                      lr=0.001, 
                      momentum=0.9)

N_EPOCHS = 10 
for epoch in range(N_EPOCHS):
    print(epoch)
    epoch_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device) 
        labels = labels.to(device)

        #optimizer.zero_grad()   # forget errors of previous pass, let's start fresh

        outputs = model(inputs)
        loss = criterion(outputs, labels)   # compute loss
        
        loss.backward()         # backpropegation; compute gradient
        
        optimizer.step()        # adjust parameters based on gradient 

        epoch_loss += loss.item()
        print(epoch_loss)
    print("Epoch: {} Loss: {}".format(epoch, 
                  epoch_loss/len(train_loader)))